<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="img/ntnu_logo.png" WIDTH=200 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Exercise 7:</ins> Multi-objective optimization
    
*Developed by David Steffelbauer*

    
<hr style="border:0.2px solid black"> </hr>

In [ ]:
# install additional packages

!pip install wntr  # https://wntr.readthedocs.io/en/latest/
!pip install deap  # https://deap.readthedocs.io

In [ ]:
# import packages and functions

import numpy as np
import wntr
import pandas as pd
import random
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
# Two Loop Network

# import network with wntr
wn = wntr.network.WaterNetworkModel('TLN.inp')  

# generate lookup table for shoowing which pipes are attached to which nodes
node2pipe = pd.concat([wn.query_link_attribute('start_node_name'), wn.query_link_attribute('end_node_name')])  

In [ ]:
# diameter options for the optimisation problem (factor 25.4 transforms inch to mm)
option_diam = np.asarray([1, 2, 3, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]) * 25.4

# costs associated to the diameters
unit_cost = np.asarray([2, 5, 8, 11, 16, 23, 32, 50, 60, 90, 130, 170, 300, 550])

# minimum requested pressure
Hreq=30

In [ ]:
def cost(genome):
    """
    calculate the costs from a genome
    """
    total_cost = 0.0
    
    # sum up the costs associated with a certain gene (factor 1000 is the length of the pipe, which is the same for all pipes)
    for gene in genome:
        total_cost += unit_cost[gene] * 1000
    return total_cost,

In [ ]:
def resilience(genome):
    """
    calculate the resilience from a genome
    """
    diameters = np.asarray([option_diam[ii] for ii in genome])

    for ii, pname in enumerate(wn.pipe_name_list):
        p = wn.get_link(pname)
        p.diameter = diameters[ii]

    Cj = dict()
    for nodename in wn.junction_name_list:

        neighbors = node2pipe[node2pipe == nodename].index
        npj = len(neighbors)
        n_diams = np.asarray([wn.get_link(p).diameter for p in neighbors])
        Cj[nodename] = np.sum(n_diams) / (npj * np.max(n_diams))

    sim = wntr.sim.EpanetSimulator(wn)
    result = sim.run_sim()

    Cj = pd.Series(Cj)
    Hj = result.node['pressure'][wn.junction_name_list].loc[0]
    Qj = result.node['demand'][wn.junction_name_list].loc[0]

    I = np.sum((Cj * Qj * (Hj - Hreq))) / np.sum(Qj * Hreq)
    return I,

In [ ]:
NBR_PIPES = len(wn.pipe_name_list)

NGEN = 30 # number of generations
MU = 200  # popsize

CXPB = 0.8
MUTPB = 0.2


In [ ]:
creator.create("Fitness", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("attr_ind", random.randint, 0, len(option_diam)-1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_ind, n=NBR_PIPES)

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.register("evaluate", cost)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(option_diam)-1, indpb=1.0/NBR_PIPES)
toolbox.register("select", tools.selTournament, tournsize=2)

In [ ]:
def soo():

    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1) # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=stats, halloffame=hof, verbose=True)

    log = pd.DataFrame(log)
    log = log.set_index('gen')

    return hof, pop, log

In [ ]:
hof, pop, log = soo()

In [ ]:
print(f'Best Solution in Hall of Fame: {hof[0]} with fitness {hof[0].fitness.values[0]}')

In [ ]:
log[['min', 'avg']].plot()
plt.legend(fontsize=14)
plt.xlim((0, NGEN))
plt.xlabel('Generations', fontsize=16)
plt.ylabel('$f(\mathbf{x})$', fontsize=16)
plt.show()

## Multi-objective optimization

In [ ]:
def eval_TLN(genome):
    return *cost(genome), *resilience(genome),

In [ ]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr_ind", random.randint, 0, len(option_diam)-1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_ind, n=NBR_PIPES)


# Structure initializers
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval_TLN)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(option_diam)-1, indpb=1.0/NBR_PIPES)
toolbox.register("select", tools.selNSGA2)

In [ ]:
def moo(seed=None):
    random.seed(seed)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min", "max"

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # This is just to assign the crowding distance to the individuals
    # no actual selection is done
    pop = toolbox.select(pop, len(pop))

    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        # Vary the population
        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]

        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= CXPB:
                toolbox.mate(ind1, ind2)

            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            del ind1.fitness.values, ind2.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
        hof.update(pop)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        print(logbook.stream)

    return hof, pop, logbook

In [ ]:
hof, pop, log = moo()

In [ ]:
print(hof)

In [ ]:
xhof, yhof = zip(*[ind.fitness.values for ind in hof])
plt.plot(xhof, yhof, label='Pareto Front')
x, y = zip(*[ind.fitness.values for ind in pop])
plt.scatter(x, y, color='k', label='Population')
plt.xlabel('Cost', fontsize=16)
plt.ylabel('Resilience', fontsize=16)
plt.legend(fontsize=14)
plt.show()